In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR


from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor,RandomForestRegressor,StackingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings('ignore')

### Load dataset 

In [2]:
df = pd.read_csv('../week1/data/train.csv')

### Split train, valid 

In [31]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

### Now use your train_df for training and test_df for evaluation.
Note: do not use test_df for training

In [32]:
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
254,255,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,145000
1066,1067,60,RL,59.0,7837,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,178000
638,639,30,RL,67.0,8777,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2008,WD,Normal,85000
799,800,50,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal,175000
380,381,50,RL,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,127000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1096,20,RL,78.0,9317,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,176432
1130,1131,50,RL,65.0,7804,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,12,2009,WD,Normal,135000
1294,1295,20,RL,60.0,8172,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,115000
860,861,50,RL,55.0,7642,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,6,2007,WD,Normal,189950


In [33]:
X_train, y_train = train_df.drop(columns='SalePrice'),train_df['SalePrice']
X_test, y_test = test_df.drop(columns='SalePrice'),test_df['SalePrice']

In [34]:
X_train.shape

(1168, 80)

In [35]:
X_test.shape

(292, 80)

In [36]:
y_train

254     145000
1066    178000
638      85000
799     175000
380     127000
         ...  
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: SalePrice, Length: 1168, dtype: int64

In [37]:
drop_features = []
def preprocess():
    #If feature missing more than 60% -> drop
    miss_feature = []
    for i in X_train:
        if(X_train.shape[0] - X_train[i].isna().sum() != 0):
            miss_feature.append(i)
    df_new_train = X_train
    for i in miss_feature:
        if(X_train[i].isna().sum()/X_train[i].shape[0] >= 0.6):
            df_new_train = df_new_train.drop([i],axis=1)
            drop_features.append(i)
            print(i)
            print(X_train[i].isna().sum()/X_train[i].shape[0])

    #drop feature id, because haven't much information 
    drop_features.append('Id')
    df_new_train = df_new_train.drop(['Id'],axis=1) 
    
    #test
    df_new_test = X_test
    for i in drop_features:
        df_new_test = df_new_test.drop([i],axis=1)
    
        
    # fill missing values = mean or mode
    for i in df_new_train:
        if(df_new_train[i].dtype == 'object'):
            df_new_train[i] = df_new_train[i].fillna(df_new_train[i].mode()[0])
            df_new_test[i] = df_new_test[i].fillna(df_new_train[i].mode()[0]) # test
        else :
            df_new_train[i] = df_new_train[i].fillna(df_new_train[i].mean())
            df_new_test[i] = df_new_test[i].fillna(df_new_train[i].mean()) #test
            
    return df_new_train,df_new_test

In [38]:
df_new_train,df_new_test = preprocess()

Alley
0.9366438356164384
PoolQC
0.9948630136986302
Fence
0.800513698630137
MiscFeature
0.9606164383561644


In [39]:
print(df_new_test.shape)
print(df_new_train.shape)

(292, 75)
(1168, 75)


In [43]:
def encoder(scaler = 0,degree = 0):
    #init
    Ordinal_Encoder = ['LotShape', 'LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','HeatingQC',
                  'KitchenQual','FireplaceQu','GarageQual','GarageCond']
    Onehot_Encoder = []
    number_feature = []
    for i in df_new_train:
        if(df_new_train[i].dtype == 'object' and i not in Ordinal_Encoder and i not in drop_features):
            Onehot_Encoder.append(i)
        if(df_new_train[i].dtype != 'object'):
            number_feature.append(i)

    # ordinal Encoder
    oe = OrdinalEncoder()
    OrdEncoder_train = oe.fit_transform(df_new_train[Ordinal_Encoder])
    OrdEncoder_test = oe.transform(df_new_test[Ordinal_Encoder])
    
    #one hot encoder
    ohe = OneHotEncoder(handle_unknown='ignore')
    OneEncoder_train = ohe.fit_transform(df_new_train[Onehot_Encoder]).toarray()
    OneEncoder_test = ohe.transform(df_new_test[Onehot_Encoder]).toarray()
    
    
    # concat
    train = np.concatenate((OrdEncoder_train,OneEncoder_train), axis=1)
    train = np.concatenate((train,np.array(df_new_train[number_feature])), axis=1)
    test = np.concatenate((OrdEncoder_test,OneEncoder_test), axis=1)
    test = np.concatenate((test,np.array(df_new_test[number_feature])), axis=1)
    
    #Standard Scaler
    if(scaler != 0):
        ss = StandardScaler()
        train = ss.fit_transform(train)
        test = ss.transform(test)
    
    #Polynomial Regression
    if(degree > 0):
        poly_reg = PolynomialFeatures(degree=degree)
        train = poly_reg.fit_transform(train)
        test = poly_reg.transform(test)
    
    #return test,train
    return train,test

In [44]:
def model(train, test):
    model1 = LinearRegression()
    model1.fit(train,y_train)
    print('LinearRegression: ',end='')
    print(mean_squared_error(y_test, model1.predict(test), squared=False))
    
    model2 = SVR()
    model2.fit(train,y_train)
    print('SVR: ',end='')
    print(mean_squared_error(y_test, model2.predict(test), squared=False))
    
    model3 = Ridge()
    model3.fit(train,y_train)
    print('Ridge: ',end='')
    print(mean_squared_error(y_test, model3.predict(test), squared=False))
    
    model4 = Lasso()
    model4.fit(train,y_train)
    print('Lasso: ',end='')
    print(mean_squared_error(y_test, model4.predict(test), squared=False))
    
    model6 = LGBMRegressor()
    model6.fit(train,y_train)
    print('LGBMRegressor: ',end='')
    print(mean_squared_error(y_test, model6.predict(test), squared=False))
    
    model7 = RandomForestRegressor()
    model7.fit(train,y_train)
    print('RandomForestRegressor: ',end='')
    print(mean_squared_error(y_test, model7.predict(test), squared=False))
    
    model5 = CatBoostRegressor()
    model5.fit(train,y_train)
    print('CatBoostRegressor: ',end='')
    print(mean_squared_error(y_test, model5.predict(test), squared=False))

In [45]:
train,test = encoder()
model(train,test)

LinearRegression: 1227003.9020006363
SVR: 88653.02105186372
Ridge: 30658.372542634355
Lasso: 29479.740762735906
LGBMRegressor: 29675.58539690644
RandomForestRegressor: 29279.930866446597
Learning rate set to 0.040124
0:	learn: 75303.7426967	total: 4.48ms	remaining: 4.47s
1:	learn: 73575.3956012	total: 8.84ms	remaining: 4.41s
2:	learn: 71722.9144612	total: 13.2ms	remaining: 4.37s
3:	learn: 70033.8231191	total: 17.4ms	remaining: 4.34s
4:	learn: 68312.9975773	total: 21.6ms	remaining: 4.29s
5:	learn: 66800.0626965	total: 25.8ms	remaining: 4.27s
6:	learn: 65197.7255418	total: 30ms	remaining: 4.25s
7:	learn: 63706.4232338	total: 34.2ms	remaining: 4.24s
8:	learn: 62233.4227708	total: 38.6ms	remaining: 4.25s
9:	learn: 60819.7728303	total: 42.6ms	remaining: 4.22s
10:	learn: 59475.2948358	total: 46.8ms	remaining: 4.21s
11:	learn: 58301.3539842	total: 55ms	remaining: 4.53s
12:	learn: 57025.9811731	total: 62.5ms	remaining: 4.75s
13:	learn: 55853.2653197	total: 68.6ms	remaining: 4.83s
14:	learn: 54

185:	learn: 17609.5299147	total: 860ms	remaining: 3.76s
186:	learn: 17575.9852150	total: 865ms	remaining: 3.76s
187:	learn: 17532.4139252	total: 870ms	remaining: 3.76s
188:	learn: 17482.0217589	total: 876ms	remaining: 3.76s
189:	learn: 17444.8074081	total: 880ms	remaining: 3.75s
190:	learn: 17395.5710888	total: 885ms	remaining: 3.75s
191:	learn: 17382.5527175	total: 893ms	remaining: 3.76s
192:	learn: 17333.9480336	total: 899ms	remaining: 3.76s
193:	learn: 17310.6007090	total: 902ms	remaining: 3.75s
194:	learn: 17267.4870935	total: 906ms	remaining: 3.74s
195:	learn: 17224.7048592	total: 913ms	remaining: 3.74s
196:	learn: 17178.4184567	total: 917ms	remaining: 3.74s
197:	learn: 17148.5962195	total: 921ms	remaining: 3.73s
198:	learn: 17105.9422726	total: 925ms	remaining: 3.72s
199:	learn: 17092.2016846	total: 929ms	remaining: 3.72s
200:	learn: 17078.5390711	total: 932ms	remaining: 3.71s
201:	learn: 17034.8368104	total: 936ms	remaining: 3.7s
202:	learn: 16990.5108922	total: 940ms	remaining:

357:	learn: 12681.7245539	total: 1.64s	remaining: 2.94s
358:	learn: 12669.6405676	total: 1.64s	remaining: 2.93s
359:	learn: 12644.2519531	total: 1.65s	remaining: 2.93s
360:	learn: 12630.4040435	total: 1.65s	remaining: 2.93s
361:	learn: 12612.6659547	total: 1.66s	remaining: 2.92s
362:	learn: 12581.4788538	total: 1.67s	remaining: 2.92s
363:	learn: 12552.1499738	total: 1.67s	remaining: 2.92s
364:	learn: 12524.5543671	total: 1.68s	remaining: 2.92s
365:	learn: 12504.5953740	total: 1.68s	remaining: 2.91s
366:	learn: 12480.7141571	total: 1.68s	remaining: 2.9s
367:	learn: 12462.0801611	total: 1.69s	remaining: 2.9s
368:	learn: 12444.5828830	total: 1.69s	remaining: 2.9s
369:	learn: 12423.3891141	total: 1.7s	remaining: 2.89s
370:	learn: 12407.7536750	total: 1.7s	remaining: 2.88s
371:	learn: 12390.2988755	total: 1.71s	remaining: 2.88s
372:	learn: 12369.4594518	total: 1.71s	remaining: 2.87s
373:	learn: 12353.1892424	total: 1.71s	remaining: 2.87s
374:	learn: 12331.2133765	total: 1.72s	remaining: 2.8

538:	learn: 9862.9490429	total: 2.42s	remaining: 2.07s
539:	learn: 9858.9252551	total: 2.43s	remaining: 2.07s
540:	learn: 9855.5315460	total: 2.44s	remaining: 2.07s
541:	learn: 9845.6355962	total: 2.45s	remaining: 2.07s
542:	learn: 9830.0482818	total: 2.46s	remaining: 2.07s
543:	learn: 9820.0383996	total: 2.46s	remaining: 2.06s
544:	learn: 9806.0260421	total: 2.46s	remaining: 2.06s
545:	learn: 9791.1706508	total: 2.47s	remaining: 2.05s
546:	learn: 9785.2517442	total: 2.47s	remaining: 2.05s
547:	learn: 9769.0354162	total: 2.48s	remaining: 2.04s
548:	learn: 9750.4510364	total: 2.48s	remaining: 2.04s
549:	learn: 9736.0763347	total: 2.49s	remaining: 2.03s
550:	learn: 9723.3311699	total: 2.49s	remaining: 2.03s
551:	learn: 9704.4454222	total: 2.5s	remaining: 2.03s
552:	learn: 9690.1475051	total: 2.5s	remaining: 2.02s
553:	learn: 9675.0679151	total: 2.51s	remaining: 2.02s
554:	learn: 9652.3823798	total: 2.51s	remaining: 2.01s
555:	learn: 9643.1613226	total: 2.52s	remaining: 2.01s
556:	learn: 

714:	learn: 8046.3073505	total: 3.2s	remaining: 1.28s
715:	learn: 8043.5163955	total: 3.21s	remaining: 1.27s
716:	learn: 8042.5384383	total: 3.21s	remaining: 1.27s
717:	learn: 8034.2306415	total: 3.22s	remaining: 1.26s
718:	learn: 8022.4416588	total: 3.22s	remaining: 1.26s
719:	learn: 8014.4193761	total: 3.23s	remaining: 1.25s
720:	learn: 8003.0913317	total: 3.23s	remaining: 1.25s
721:	learn: 7996.8060585	total: 3.24s	remaining: 1.25s
722:	learn: 7990.0275994	total: 3.24s	remaining: 1.24s
723:	learn: 7983.8784774	total: 3.25s	remaining: 1.24s
724:	learn: 7972.3233491	total: 3.25s	remaining: 1.23s
725:	learn: 7965.4354150	total: 3.26s	remaining: 1.23s
726:	learn: 7954.7949970	total: 3.26s	remaining: 1.22s
727:	learn: 7950.6017200	total: 3.26s	remaining: 1.22s
728:	learn: 7943.6223892	total: 3.27s	remaining: 1.22s
729:	learn: 7933.2251831	total: 3.27s	remaining: 1.21s
730:	learn: 7923.9440371	total: 3.28s	remaining: 1.21s
731:	learn: 7906.4985764	total: 3.28s	remaining: 1.2s
732:	learn: 

869:	learn: 6840.5981585	total: 3.99s	remaining: 596ms
870:	learn: 6839.9032607	total: 3.99s	remaining: 591ms
871:	learn: 6828.3190087	total: 4s	remaining: 587ms
872:	learn: 6819.8807431	total: 4s	remaining: 582ms
873:	learn: 6817.4707989	total: 4.01s	remaining: 578ms
874:	learn: 6812.2384850	total: 4.01s	remaining: 574ms
875:	learn: 6807.8692285	total: 4.02s	remaining: 569ms
876:	learn: 6795.1598324	total: 4.02s	remaining: 564ms
877:	learn: 6785.1797662	total: 4.03s	remaining: 560ms
878:	learn: 6778.4702385	total: 4.03s	remaining: 555ms
879:	learn: 6777.8641855	total: 4.04s	remaining: 551ms
880:	learn: 6769.3073872	total: 4.04s	remaining: 546ms
881:	learn: 6760.8387553	total: 4.04s	remaining: 541ms
882:	learn: 6757.1605054	total: 4.05s	remaining: 537ms
883:	learn: 6755.3553021	total: 4.05s	remaining: 532ms
884:	learn: 6747.0353515	total: 4.06s	remaining: 527ms
885:	learn: 6737.5243572	total: 4.06s	remaining: 523ms
886:	learn: 6733.2939765	total: 4.07s	remaining: 518ms
887:	learn: 6729

In [47]:
train,test = encoder(degree=2)
model(train,test)

KeyboardInterrupt: 

In [340]:
train,test = encoder()
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
train = svd.fit_transform(train)
test = svd.transform(test)

In [341]:
model(train,test)

LinearRegression: 34495.80704210149
SVR: 88653.09406842804
Ridge: 34429.96680308513
Lasso: 34496.02588933019
LGBMRegressor: 35309.565235172246
RandomForestRegressor: 38910.4893339053
Learning rate set to 0.040124
0:	learn: 75601.2851080	total: 20.9ms	remaining: 20.8s
1:	learn: 74198.8879905	total: 38.9ms	remaining: 19.4s
2:	learn: 72689.9961200	total: 58.2ms	remaining: 19.3s
3:	learn: 71513.6712489	total: 78.3ms	remaining: 19.5s
4:	learn: 70247.1734695	total: 99.9ms	remaining: 19.9s
5:	learn: 68935.7271788	total: 120ms	remaining: 19.9s
6:	learn: 67703.0896364	total: 139ms	remaining: 19.7s
7:	learn: 66588.4424654	total: 157ms	remaining: 19.4s
8:	learn: 65374.4334750	total: 179ms	remaining: 19.7s
9:	learn: 64096.3133190	total: 199ms	remaining: 19.7s
10:	learn: 62917.4196864	total: 217ms	remaining: 19.5s
11:	learn: 61848.4013627	total: 237ms	remaining: 19.5s
12:	learn: 60674.5625164	total: 258ms	remaining: 19.6s
13:	learn: 59540.5638524	total: 278ms	remaining: 19.6s
14:	learn: 58477.40963

150:	learn: 21487.0405525	total: 3.3s	remaining: 18.6s
151:	learn: 21415.5678641	total: 3.32s	remaining: 18.5s
152:	learn: 21321.9502841	total: 3.35s	remaining: 18.5s
153:	learn: 21220.6603643	total: 3.37s	remaining: 18.5s
154:	learn: 21156.7226004	total: 3.39s	remaining: 18.5s
155:	learn: 21054.2165658	total: 3.42s	remaining: 18.5s
156:	learn: 20956.4940690	total: 3.44s	remaining: 18.5s
157:	learn: 20871.8136267	total: 3.46s	remaining: 18.5s
158:	learn: 20789.0410343	total: 3.49s	remaining: 18.5s
159:	learn: 20722.0465858	total: 3.52s	remaining: 18.5s
160:	learn: 20637.9984479	total: 3.54s	remaining: 18.4s
161:	learn: 20531.0519082	total: 3.56s	remaining: 18.4s
162:	learn: 20460.2406710	total: 3.58s	remaining: 18.4s
163:	learn: 20397.7867350	total: 3.59s	remaining: 18.3s
164:	learn: 20297.0086351	total: 3.61s	remaining: 18.3s
165:	learn: 20227.4695040	total: 3.63s	remaining: 18.3s
166:	learn: 20149.2587400	total: 3.66s	remaining: 18.2s
167:	learn: 20077.8102167	total: 3.68s	remaining:

298:	learn: 13138.8715084	total: 7.4s	remaining: 17.4s
299:	learn: 13108.8637249	total: 7.43s	remaining: 17.3s
300:	learn: 13062.5696265	total: 7.47s	remaining: 17.3s
301:	learn: 13030.5543799	total: 7.5s	remaining: 17.3s
302:	learn: 12983.1432359	total: 7.53s	remaining: 17.3s
303:	learn: 12947.7342764	total: 7.56s	remaining: 17.3s
304:	learn: 12919.6260815	total: 7.59s	remaining: 17.3s
305:	learn: 12876.3924488	total: 7.62s	remaining: 17.3s
306:	learn: 12856.7757840	total: 7.64s	remaining: 17.2s
307:	learn: 12841.2754239	total: 7.66s	remaining: 17.2s
308:	learn: 12791.3589817	total: 7.68s	remaining: 17.2s
309:	learn: 12760.7231294	total: 7.71s	remaining: 17.2s
310:	learn: 12733.2351669	total: 7.73s	remaining: 17.1s
311:	learn: 12693.2096088	total: 7.75s	remaining: 17.1s
312:	learn: 12676.8613722	total: 7.77s	remaining: 17.1s
313:	learn: 12647.5455229	total: 7.79s	remaining: 17s
314:	learn: 12614.6690518	total: 7.81s	remaining: 17s
315:	learn: 12601.6668244	total: 7.84s	remaining: 17s


450:	learn: 8907.3953606	total: 10.9s	remaining: 13.2s
451:	learn: 8901.8066824	total: 10.9s	remaining: 13.2s
452:	learn: 8877.2214043	total: 10.9s	remaining: 13.2s
453:	learn: 8856.4047333	total: 10.9s	remaining: 13.2s
454:	learn: 8823.8767929	total: 11s	remaining: 13.1s
455:	learn: 8807.2275351	total: 11s	remaining: 13.1s
456:	learn: 8792.7112654	total: 11s	remaining: 13.1s
457:	learn: 8764.2066425	total: 11s	remaining: 13.1s
458:	learn: 8749.6683512	total: 11.1s	remaining: 13s
459:	learn: 8725.7777759	total: 11.1s	remaining: 13s
460:	learn: 8705.7651354	total: 11.1s	remaining: 13s
461:	learn: 8684.1224570	total: 11.1s	remaining: 13s
462:	learn: 8656.3838961	total: 11.2s	remaining: 13s
463:	learn: 8632.3239897	total: 11.2s	remaining: 12.9s
464:	learn: 8603.0210386	total: 11.2s	remaining: 12.9s
465:	learn: 8596.9022756	total: 11.2s	remaining: 12.9s
466:	learn: 8590.4096348	total: 11.3s	remaining: 12.8s
467:	learn: 8563.6682238	total: 11.3s	remaining: 12.8s
468:	learn: 8540.4888843	tot

610:	learn: 6064.6758190	total: 14.2s	remaining: 9.07s
611:	learn: 6048.8913034	total: 14.3s	remaining: 9.05s
612:	learn: 6038.2884490	total: 14.3s	remaining: 9.03s
613:	learn: 6016.2933646	total: 14.3s	remaining: 9s
614:	learn: 6006.8517494	total: 14.3s	remaining: 8.97s
615:	learn: 5992.1871714	total: 14.4s	remaining: 8.95s
616:	learn: 5969.6148988	total: 14.4s	remaining: 8.93s
617:	learn: 5950.1827551	total: 14.4s	remaining: 8.9s
618:	learn: 5940.2408059	total: 14.4s	remaining: 8.88s
619:	learn: 5925.2845340	total: 14.5s	remaining: 8.86s
620:	learn: 5916.5814810	total: 14.5s	remaining: 8.84s
621:	learn: 5908.0534662	total: 14.5s	remaining: 8.81s
622:	learn: 5897.6143047	total: 14.5s	remaining: 8.79s
623:	learn: 5884.9570090	total: 14.5s	remaining: 8.76s
624:	learn: 5874.8199867	total: 14.6s	remaining: 8.73s
625:	learn: 5857.9762484	total: 14.6s	remaining: 8.71s
626:	learn: 5848.1481168	total: 14.6s	remaining: 8.68s
627:	learn: 5830.8802060	total: 14.6s	remaining: 8.65s
628:	learn: 58

764:	learn: 4361.7044081	total: 18.4s	remaining: 5.64s
765:	learn: 4350.6052488	total: 18.4s	remaining: 5.62s
766:	learn: 4347.0953173	total: 18.4s	remaining: 5.6s
767:	learn: 4334.3511043	total: 18.5s	remaining: 5.58s
768:	learn: 4324.3415996	total: 18.5s	remaining: 5.55s
769:	learn: 4315.9972657	total: 18.5s	remaining: 5.53s
770:	learn: 4309.2055193	total: 18.6s	remaining: 5.51s
771:	learn: 4303.1579017	total: 18.6s	remaining: 5.49s
772:	learn: 4294.7149227	total: 18.6s	remaining: 5.46s
773:	learn: 4289.3828642	total: 18.6s	remaining: 5.44s
774:	learn: 4282.4377122	total: 18.7s	remaining: 5.42s
775:	learn: 4276.9745753	total: 18.7s	remaining: 5.4s
776:	learn: 4268.4953483	total: 18.7s	remaining: 5.37s
777:	learn: 4257.6612489	total: 18.8s	remaining: 5.35s
778:	learn: 4248.7776971	total: 18.8s	remaining: 5.33s
779:	learn: 4242.7351437	total: 18.8s	remaining: 5.31s
780:	learn: 4235.2523901	total: 18.8s	remaining: 5.28s
781:	learn: 4225.1112363	total: 18.9s	remaining: 5.26s
782:	learn: 

916:	learn: 3210.4555298	total: 22.5s	remaining: 2.04s
917:	learn: 3207.9780164	total: 22.5s	remaining: 2.01s
918:	learn: 3200.1102091	total: 22.6s	remaining: 1.99s
919:	learn: 3192.6615462	total: 22.6s	remaining: 1.97s
920:	learn: 3183.5430982	total: 22.6s	remaining: 1.94s
921:	learn: 3180.3491536	total: 22.7s	remaining: 1.92s
922:	learn: 3171.2203006	total: 22.7s	remaining: 1.9s
923:	learn: 3166.6051364	total: 22.8s	remaining: 1.87s
924:	learn: 3158.5630264	total: 22.8s	remaining: 1.85s
925:	learn: 3151.2144523	total: 22.8s	remaining: 1.82s
926:	learn: 3139.2583889	total: 22.9s	remaining: 1.8s
927:	learn: 3137.7939141	total: 22.9s	remaining: 1.78s
928:	learn: 3133.5342753	total: 22.9s	remaining: 1.75s
929:	learn: 3125.3872930	total: 23s	remaining: 1.73s
930:	learn: 3121.6270277	total: 23s	remaining: 1.71s
931:	learn: 3114.2745072	total: 23.1s	remaining: 1.68s
932:	learn: 3107.3717727	total: 23.1s	remaining: 1.66s
933:	learn: 3097.1884740	total: 23.1s	remaining: 1.63s
934:	learn: 3088

In [363]:
train,test = encoder()
er = VotingRegressor([
                      ('RandomForestRegressor', RandomForestRegressor(n_estimators=300)), 
                      ('CatBoostRegressor', CatBoostRegressor()),
                      ('Lasso', Lasso()),
                      ('CatBoostRegressor2', CatBoostRegressor()),
                     ], weights=[1,2,1,1])
er.fit(train,y_train)
print(mean_squared_error(y_test, er.predict(test), squared=False))

Learning rate set to 0.040124
0:	learn: 75303.7426967	total: 4.27ms	remaining: 4.27s
1:	learn: 73575.3956012	total: 8.57ms	remaining: 4.27s
2:	learn: 71722.9144612	total: 13ms	remaining: 4.3s
3:	learn: 70033.8231191	total: 17.2ms	remaining: 4.28s
4:	learn: 68312.9975773	total: 22.2ms	remaining: 4.42s
5:	learn: 66800.0626965	total: 26.3ms	remaining: 4.36s
6:	learn: 65197.7255418	total: 30.4ms	remaining: 4.32s
7:	learn: 63706.4232338	total: 36.3ms	remaining: 4.5s
8:	learn: 62233.4227708	total: 40.8ms	remaining: 4.49s
9:	learn: 60819.7728303	total: 44.9ms	remaining: 4.45s
10:	learn: 59475.2948358	total: 49.1ms	remaining: 4.42s
11:	learn: 58301.3539842	total: 53.5ms	remaining: 4.41s
12:	learn: 57025.9811731	total: 57.8ms	remaining: 4.38s
13:	learn: 55853.2653197	total: 61.9ms	remaining: 4.36s
14:	learn: 54656.6746294	total: 66.2ms	remaining: 4.35s
15:	learn: 53548.0073731	total: 70.5ms	remaining: 4.33s
16:	learn: 52408.8573187	total: 74.7ms	remaining: 4.32s
17:	learn: 51297.6715207	total: 

167:	learn: 18236.4892638	total: 789ms	remaining: 3.91s
168:	learn: 18199.5215238	total: 795ms	remaining: 3.91s
169:	learn: 18182.6801213	total: 801ms	remaining: 3.91s
170:	learn: 18130.2004547	total: 807ms	remaining: 3.91s
171:	learn: 18107.6641165	total: 813ms	remaining: 3.91s
172:	learn: 18081.8818769	total: 818ms	remaining: 3.91s
173:	learn: 18035.1784031	total: 823ms	remaining: 3.9s
174:	learn: 17987.8209398	total: 827ms	remaining: 3.9s
175:	learn: 17961.2862258	total: 831ms	remaining: 3.89s
176:	learn: 17907.7687165	total: 835ms	remaining: 3.88s
177:	learn: 17869.6092229	total: 839ms	remaining: 3.88s
178:	learn: 17813.5374091	total: 845ms	remaining: 3.88s
179:	learn: 17761.9685919	total: 849ms	remaining: 3.87s
180:	learn: 17711.5766537	total: 854ms	remaining: 3.86s
181:	learn: 17689.3815877	total: 858ms	remaining: 3.86s
182:	learn: 17659.8585436	total: 862ms	remaining: 3.85s
183:	learn: 17645.8539648	total: 866ms	remaining: 3.84s
184:	learn: 17631.2646328	total: 871ms	remaining: 

342:	learn: 13009.7271926	total: 1.56s	remaining: 3s
343:	learn: 13006.8473988	total: 1.57s	remaining: 2.99s
344:	learn: 12986.3632598	total: 1.57s	remaining: 2.99s
345:	learn: 12966.3153513	total: 1.58s	remaining: 2.98s
346:	learn: 12942.0147857	total: 1.58s	remaining: 2.98s
347:	learn: 12918.5293914	total: 1.59s	remaining: 2.98s
348:	learn: 12892.2306530	total: 1.6s	remaining: 2.98s
349:	learn: 12870.2920245	total: 1.6s	remaining: 2.98s
350:	learn: 12837.1951353	total: 1.61s	remaining: 2.97s
351:	learn: 12816.4665972	total: 1.61s	remaining: 2.97s
352:	learn: 12796.5872819	total: 1.62s	remaining: 2.96s
353:	learn: 12786.0039150	total: 1.62s	remaining: 2.96s
354:	learn: 12757.6797500	total: 1.63s	remaining: 2.95s
355:	learn: 12722.5677136	total: 1.63s	remaining: 2.95s
356:	learn: 12708.0247938	total: 1.63s	remaining: 2.94s
357:	learn: 12681.7245539	total: 1.64s	remaining: 2.94s
358:	learn: 12669.6405676	total: 1.64s	remaining: 2.93s
359:	learn: 12644.2519531	total: 1.65s	remaining: 2.9

514:	learn: 10145.1967538	total: 2.34s	remaining: 2.21s
515:	learn: 10127.9456372	total: 2.35s	remaining: 2.2s
516:	learn: 10124.2336981	total: 2.35s	remaining: 2.2s
517:	learn: 10108.0595440	total: 2.36s	remaining: 2.19s
518:	learn: 10096.5632717	total: 2.36s	remaining: 2.19s
519:	learn: 10074.9803325	total: 2.37s	remaining: 2.19s
520:	learn: 10069.2108262	total: 2.38s	remaining: 2.18s
521:	learn: 10051.1007662	total: 2.38s	remaining: 2.18s
522:	learn: 10048.7969716	total: 2.38s	remaining: 2.17s
523:	learn: 10037.4340523	total: 2.39s	remaining: 2.17s
524:	learn: 10029.8315476	total: 2.39s	remaining: 2.17s
525:	learn: 10008.9021243	total: 2.4s	remaining: 2.16s
526:	learn: 9995.2591996	total: 2.4s	remaining: 2.16s
527:	learn: 9991.5020439	total: 2.41s	remaining: 2.15s
528:	learn: 9989.5751218	total: 2.41s	remaining: 2.15s
529:	learn: 9979.4950084	total: 2.42s	remaining: 2.14s
530:	learn: 9965.6907126	total: 2.42s	remaining: 2.14s
531:	learn: 9956.4093602	total: 2.42s	remaining: 2.13s
53

685:	learn: 8343.4746885	total: 3.13s	remaining: 1.43s
686:	learn: 8331.0209699	total: 3.14s	remaining: 1.43s
687:	learn: 8314.5723427	total: 3.15s	remaining: 1.43s
688:	learn: 8301.6326765	total: 3.16s	remaining: 1.43s
689:	learn: 8288.8759227	total: 3.17s	remaining: 1.42s
690:	learn: 8286.4797483	total: 3.17s	remaining: 1.42s
691:	learn: 8283.9923875	total: 3.17s	remaining: 1.41s
692:	learn: 8268.1600567	total: 3.18s	remaining: 1.41s
693:	learn: 8267.2119598	total: 3.18s	remaining: 1.4s
694:	learn: 8256.2257423	total: 3.19s	remaining: 1.4s
695:	learn: 8249.7259099	total: 3.19s	remaining: 1.39s
696:	learn: 8242.7705533	total: 3.2s	remaining: 1.39s
697:	learn: 8227.4761808	total: 3.2s	remaining: 1.38s
698:	learn: 8221.4735492	total: 3.21s	remaining: 1.38s
699:	learn: 8219.9752116	total: 3.21s	remaining: 1.38s
700:	learn: 8212.5105803	total: 3.21s	remaining: 1.37s
701:	learn: 8205.3663362	total: 3.22s	remaining: 1.36s
702:	learn: 8198.6040789	total: 3.22s	remaining: 1.36s
703:	learn: 81

...

217:	learn: 16593.0166082	total: 983ms	remaining: 3.53s
218:	learn: 16583.7557847	total: 988ms	remaining: 3.52s
219:	learn: 16552.8944294	total: 993ms	remaining: 3.52s
220:	learn: 16511.0720994	total: 999ms	remaining: 3.52s
221:	learn: 16482.6773510	total: 1s	remaining: 3.52s
222:	learn: 16455.5427377	total: 1.01s	remaining: 3.52s
223:	learn: 16414.4418901	total: 1.01s	remaining: 3.51s
224:	learn: 16401.4164098	total: 1.02s	remaining: 3.5s
225:	learn: 16366.4693791	total: 1.02s	remaining: 3.51s
226:	learn: 16337.0264555	total: 1.03s	remaining: 3.5s
227:	learn: 16292.6424399	total: 1.03s	remaining: 3.5s
228:	learn: 16281.8681598	total: 1.04s	remaining: 3.49s
229:	learn: 16253.7348871	total: 1.04s	remaining: 3.48s
230:	learn: 16245.0626352	total: 1.04s	remaining: 3.47s
231:	learn: 16206.4744806	total: 1.05s	remaining: 3.47s
232:	learn: 16154.7491503	total: 1.05s	remaining: 3.46s
233:	learn: 16120.8714155	total: 1.05s	remaining: 3.46s
234:	learn: 16080.9338124	total: 1.06s	remaining: 3.45

390:	learn: 12017.2828716	total: 1.76s	remaining: 2.75s
391:	learn: 11993.9219519	total: 1.77s	remaining: 2.75s
392:	learn: 11977.3000455	total: 1.78s	remaining: 2.75s
393:	learn: 11969.8112112	total: 1.78s	remaining: 2.74s
394:	learn: 11953.0779665	total: 1.79s	remaining: 2.74s
395:	learn: 11928.4736954	total: 1.79s	remaining: 2.73s
396:	learn: 11904.6196053	total: 1.8s	remaining: 2.73s
397:	learn: 11883.3241675	total: 1.8s	remaining: 2.73s
398:	learn: 11870.5938797	total: 1.81s	remaining: 2.72s
399:	learn: 11854.4419729	total: 1.81s	remaining: 2.72s
400:	learn: 11852.3083432	total: 1.82s	remaining: 2.71s
401:	learn: 11836.0142843	total: 1.82s	remaining: 2.71s
402:	learn: 11820.3567118	total: 1.83s	remaining: 2.71s
403:	learn: 11809.8750685	total: 1.83s	remaining: 2.7s
404:	learn: 11795.3798423	total: 1.83s	remaining: 2.69s
405:	learn: 11770.6869326	total: 1.84s	remaining: 2.69s
406:	learn: 11748.8230563	total: 1.84s	remaining: 2.69s
407:	learn: 11735.0945502	total: 1.85s	remaining: 2

560:	learn: 9572.7785800	total: 2.54s	remaining: 1.99s
561:	learn: 9557.7897504	total: 2.54s	remaining: 1.98s
562:	learn: 9547.4482241	total: 2.55s	remaining: 1.98s
563:	learn: 9541.3281935	total: 2.55s	remaining: 1.97s
564:	learn: 9525.4040966	total: 2.56s	remaining: 1.97s
565:	learn: 9515.5172046	total: 2.56s	remaining: 1.97s
566:	learn: 9503.0433265	total: 2.57s	remaining: 1.96s
567:	learn: 9494.6687943	total: 2.57s	remaining: 1.96s
568:	learn: 9481.3174836	total: 2.58s	remaining: 1.96s
569:	learn: 9463.7910987	total: 2.59s	remaining: 1.95s
570:	learn: 9451.3795127	total: 2.59s	remaining: 1.95s
571:	learn: 9434.9569502	total: 2.6s	remaining: 1.94s
572:	learn: 9423.3549978	total: 2.6s	remaining: 1.94s
573:	learn: 9405.1255754	total: 2.61s	remaining: 1.93s
574:	learn: 9386.8695655	total: 2.61s	remaining: 1.93s
575:	learn: 9374.1838713	total: 2.62s	remaining: 1.92s
576:	learn: 9361.2474634	total: 2.62s	remaining: 1.92s
577:	learn: 9343.7427041	total: 2.62s	remaining: 1.92s
578:	learn: 

729:	learn: 7933.2251831	total: 3.31s	remaining: 1.22s
730:	learn: 7923.9440371	total: 3.32s	remaining: 1.22s
731:	learn: 7906.4985764	total: 3.32s	remaining: 1.22s
732:	learn: 7897.8615232	total: 3.33s	remaining: 1.21s
733:	learn: 7896.8964321	total: 3.33s	remaining: 1.21s
734:	learn: 7884.5527779	total: 3.34s	remaining: 1.2s
735:	learn: 7873.6435940	total: 3.34s	remaining: 1.2s
736:	learn: 7865.1990261	total: 3.35s	remaining: 1.19s
737:	learn: 7850.8117468	total: 3.35s	remaining: 1.19s
738:	learn: 7838.7050952	total: 3.35s	remaining: 1.19s
739:	learn: 7833.4283643	total: 3.36s	remaining: 1.18s
740:	learn: 7827.3992781	total: 3.37s	remaining: 1.18s
741:	learn: 7821.7490544	total: 3.37s	remaining: 1.17s
742:	learn: 7820.6485709	total: 3.38s	remaining: 1.17s
743:	learn: 7815.9668755	total: 3.38s	remaining: 1.16s
744:	learn: 7799.6624680	total: 3.38s	remaining: 1.16s
745:	learn: 7791.4214607	total: 3.39s	remaining: 1.15s
746:	learn: 7790.6631426	total: 3.39s	remaining: 1.15s
747:	learn: 

893:	learn: 6682.7112978	total: 4.3s	remaining: 510ms
894:	learn: 6682.0076149	total: 4.31s	remaining: 506ms
895:	learn: 6671.8570567	total: 4.31s	remaining: 501ms
896:	learn: 6671.2315801	total: 4.32s	remaining: 496ms
897:	learn: 6662.9641221	total: 4.33s	remaining: 492ms
898:	learn: 6655.8693721	total: 4.33s	remaining: 487ms
899:	learn: 6649.6832759	total: 4.35s	remaining: 483ms
900:	learn: 6640.9003457	total: 4.35s	remaining: 478ms
901:	learn: 6625.2508103	total: 4.36s	remaining: 474ms
902:	learn: 6622.7719699	total: 4.37s	remaining: 469ms
903:	learn: 6622.1144047	total: 4.38s	remaining: 465ms
904:	learn: 6616.7891799	total: 4.39s	remaining: 461ms
905:	learn: 6610.8228211	total: 4.39s	remaining: 456ms
906:	learn: 6599.8996485	total: 4.4s	remaining: 451ms
907:	learn: 6588.5330009	total: 4.4s	remaining: 446ms
908:	learn: 6582.6334160	total: 4.41s	remaining: 441ms
909:	learn: 6574.5204105	total: 4.41s	remaining: 436ms
910:	learn: 6571.7881222	total: 4.42s	remaining: 432ms
911:	learn: 6

In [368]:
er.score(train,y_train)

0.987979062741465

In [15]:
train,test = encoder()
sr = StackingRegressor(estimators=[
                      ('RandomForestRegressor', RandomForestRegressor(n_estimators=300)), 
                      ('CatBoostRegressor', CatBoostRegressor()),
                      ('Lasso', Lasso())
                      #('LGBMRegressor', LGBMRegressor())
                     ], final_estimator=Lasso())
sr.fit(train,y_train)
print(mean_squared_error(y_test, sr.predict(test), squared=False))

Learning rate set to 0.040124
0:	learn: 75303.7426967	total: 64.9ms	remaining: 1m 4s
1:	learn: 73575.3956012	total: 69.7ms	remaining: 34.8s
2:	learn: 71722.9144612	total: 74ms	remaining: 24.6s
3:	learn: 70033.8231191	total: 78.2ms	remaining: 19.5s
4:	learn: 68312.9975773	total: 82.3ms	remaining: 16.4s
5:	learn: 66800.0626965	total: 86.4ms	remaining: 14.3s
6:	learn: 65197.7255418	total: 90.6ms	remaining: 12.9s
7:	learn: 63706.4232338	total: 94.7ms	remaining: 11.7s
8:	learn: 62233.4227708	total: 99.1ms	remaining: 10.9s
9:	learn: 60819.7728303	total: 104ms	remaining: 10.2s
10:	learn: 59475.2948358	total: 109ms	remaining: 9.77s
11:	learn: 58301.3539842	total: 113ms	remaining: 9.29s
12:	learn: 57025.9811731	total: 117ms	remaining: 8.88s
13:	learn: 55853.2653197	total: 121ms	remaining: 8.53s
14:	learn: 54656.6746294	total: 125ms	remaining: 8.22s
15:	learn: 53548.0073731	total: 129ms	remaining: 7.95s
16:	learn: 52408.8573187	total: 133ms	remaining: 7.71s
17:	learn: 51297.6715207	total: 138ms	

156:	learn: 18788.7469764	total: 971ms	remaining: 5.21s
157:	learn: 18736.2958776	total: 980ms	remaining: 5.22s
158:	learn: 18695.6087362	total: 989ms	remaining: 5.23s
159:	learn: 18647.6612140	total: 997ms	remaining: 5.23s
160:	learn: 18590.6168358	total: 1s	remaining: 5.24s
161:	learn: 18524.0864836	total: 1.01s	remaining: 5.25s
162:	learn: 18467.3296157	total: 1.02s	remaining: 5.26s
163:	learn: 18396.7995948	total: 1.03s	remaining: 5.26s
164:	learn: 18355.8102288	total: 1.04s	remaining: 5.27s
165:	learn: 18335.2389905	total: 1.05s	remaining: 5.29s
166:	learn: 18288.6781242	total: 1.07s	remaining: 5.35s
167:	learn: 18236.4892638	total: 1.09s	remaining: 5.39s
168:	learn: 18199.5215238	total: 1.1s	remaining: 5.42s
169:	learn: 18182.6801213	total: 1.12s	remaining: 5.45s
170:	learn: 18130.2004547	total: 1.13s	remaining: 5.46s
171:	learn: 18107.6641165	total: 1.13s	remaining: 5.45s
172:	learn: 18081.8818769	total: 1.14s	remaining: 5.45s
173:	learn: 18035.1784031	total: 1.14s	remaining: 5.

330:	learn: 13239.0372938	total: 1.95s	remaining: 3.95s
331:	learn: 13208.9344010	total: 1.96s	remaining: 3.94s
332:	learn: 13191.2941098	total: 1.96s	remaining: 3.93s
333:	learn: 13171.8558713	total: 1.97s	remaining: 3.92s
334:	learn: 13155.2838900	total: 1.97s	remaining: 3.92s
335:	learn: 13126.4214939	total: 1.98s	remaining: 3.91s
336:	learn: 13106.9312686	total: 1.98s	remaining: 3.9s
337:	learn: 13097.0102101	total: 1.99s	remaining: 3.89s
338:	learn: 13073.0241130	total: 1.99s	remaining: 3.88s
339:	learn: 13060.9252991	total: 2s	remaining: 3.88s
340:	learn: 13045.6898300	total: 2s	remaining: 3.88s
341:	learn: 13013.8677645	total: 2.01s	remaining: 3.87s
342:	learn: 13009.7271926	total: 2.01s	remaining: 3.86s
343:	learn: 13006.8473988	total: 2.02s	remaining: 3.85s
344:	learn: 12986.3632598	total: 2.02s	remaining: 3.84s
345:	learn: 12966.3153513	total: 2.03s	remaining: 3.83s
346:	learn: 12942.0147857	total: 2.03s	remaining: 3.82s
347:	learn: 12918.5293914	total: 2.04s	remaining: 3.81s

505:	learn: 10221.4125045	total: 2.73s	remaining: 2.67s
506:	learn: 10206.3029088	total: 2.74s	remaining: 2.66s
507:	learn: 10204.9697376	total: 2.74s	remaining: 2.66s
508:	learn: 10200.3818958	total: 2.75s	remaining: 2.65s
509:	learn: 10192.8764155	total: 2.75s	remaining: 2.65s
510:	learn: 10175.8673840	total: 2.76s	remaining: 2.64s
511:	learn: 10168.1883388	total: 2.77s	remaining: 2.63s
512:	learn: 10153.8799158	total: 2.77s	remaining: 2.63s
513:	learn: 10150.1571456	total: 2.77s	remaining: 2.62s
514:	learn: 10145.1967538	total: 2.78s	remaining: 2.62s
515:	learn: 10127.9456372	total: 2.78s	remaining: 2.61s
516:	learn: 10124.2336981	total: 2.79s	remaining: 2.6s
517:	learn: 10108.0595440	total: 2.79s	remaining: 2.6s
518:	learn: 10096.5632717	total: 2.8s	remaining: 2.59s
519:	learn: 10074.9803325	total: 2.8s	remaining: 2.58s
520:	learn: 10069.2108262	total: 2.81s	remaining: 2.58s
521:	learn: 10051.1007662	total: 2.81s	remaining: 2.57s
522:	learn: 10048.7969716	total: 2.81s	remaining: 2.

678:	learn: 8413.1784692	total: 3.52s	remaining: 1.66s
679:	learn: 8405.0547936	total: 3.52s	remaining: 1.66s
680:	learn: 8401.6567782	total: 3.53s	remaining: 1.65s
681:	learn: 8385.2898327	total: 3.54s	remaining: 1.65s
682:	learn: 8372.7835640	total: 3.55s	remaining: 1.65s
683:	learn: 8363.7749390	total: 3.55s	remaining: 1.64s
684:	learn: 8360.0675651	total: 3.56s	remaining: 1.64s
685:	learn: 8343.4746885	total: 3.57s	remaining: 1.63s
686:	learn: 8331.0209699	total: 3.57s	remaining: 1.63s
687:	learn: 8314.5723427	total: 3.58s	remaining: 1.62s
688:	learn: 8301.6326765	total: 3.58s	remaining: 1.62s
689:	learn: 8288.8759227	total: 3.59s	remaining: 1.61s
690:	learn: 8286.4797483	total: 3.6s	remaining: 1.61s
691:	learn: 8283.9923875	total: 3.6s	remaining: 1.6s
692:	learn: 8268.1600567	total: 3.61s	remaining: 1.6s
693:	learn: 8267.2119598	total: 3.61s	remaining: 1.59s
694:	learn: 8256.2257423	total: 3.62s	remaining: 1.59s
695:	learn: 8249.7259099	total: 3.62s	remaining: 1.58s
696:	learn: 82

...

335:	learn: 13152.5339598	total: 2.01s	remaining: 3.97s
336:	learn: 13127.0899768	total: 2.01s	remaining: 3.96s
337:	learn: 13098.8332020	total: 2.02s	remaining: 3.96s
338:	learn: 13069.9137526	total: 2.03s	remaining: 3.95s
339:	learn: 13040.3875302	total: 2.04s	remaining: 3.96s
340:	learn: 13007.3090787	total: 2.05s	remaining: 3.96s
341:	learn: 12978.1524437	total: 2.05s	remaining: 3.95s
342:	learn: 12952.2649420	total: 2.06s	remaining: 3.94s
343:	learn: 12944.1557244	total: 2.06s	remaining: 3.93s
344:	learn: 12910.6605882	total: 2.07s	remaining: 3.92s
345:	learn: 12882.1404477	total: 2.07s	remaining: 3.91s
346:	learn: 12847.4213208	total: 2.08s	remaining: 3.92s
347:	learn: 12820.6793524	total: 2.09s	remaining: 3.92s
348:	learn: 12794.1571638	total: 2.09s	remaining: 3.91s
349:	learn: 12787.5222873	total: 2.1s	remaining: 3.9s
350:	learn: 12761.3897932	total: 2.1s	remaining: 3.89s
351:	learn: 12723.4271753	total: 2.11s	remaining: 3.88s
352:	learn: 12699.1447302	total: 2.11s	remaining: 3

501:	learn: 10025.9590067	total: 3s	remaining: 2.97s
502:	learn: 10005.5494013	total: 3.01s	remaining: 2.97s
503:	learn: 9984.1653154	total: 3.02s	remaining: 2.98s
504:	learn: 9962.0323737	total: 3.04s	remaining: 2.98s
505:	learn: 9955.1348568	total: 3.04s	remaining: 2.97s
506:	learn: 9935.1043718	total: 3.05s	remaining: 2.96s
507:	learn: 9911.4792491	total: 3.05s	remaining: 2.96s
508:	learn: 9893.7478476	total: 3.06s	remaining: 2.95s
509:	learn: 9882.2927559	total: 3.06s	remaining: 2.94s
510:	learn: 9880.4413510	total: 3.07s	remaining: 2.94s
511:	learn: 9878.7510237	total: 3.08s	remaining: 2.94s
512:	learn: 9877.2249885	total: 3.09s	remaining: 2.93s
513:	learn: 9865.2415023	total: 3.09s	remaining: 2.92s
514:	learn: 9857.0337342	total: 3.1s	remaining: 2.92s
515:	learn: 9847.6550536	total: 3.11s	remaining: 2.92s
516:	learn: 9830.3755291	total: 3.12s	remaining: 2.91s
517:	learn: 9817.6533301	total: 3.13s	remaining: 2.91s
518:	learn: 9799.8049690	total: 3.13s	remaining: 2.9s
519:	learn: 9

682:	learn: 7831.2188814	total: 4s	remaining: 1.86s
683:	learn: 7823.3399737	total: 4.01s	remaining: 1.85s
684:	learn: 7816.8644624	total: 4.02s	remaining: 1.85s
685:	learn: 7804.1315541	total: 4.02s	remaining: 1.84s
686:	learn: 7791.8142605	total: 4.03s	remaining: 1.84s
687:	learn: 7787.1432633	total: 4.04s	remaining: 1.83s
688:	learn: 7776.5226525	total: 4.04s	remaining: 1.82s
689:	learn: 7771.7437458	total: 4.05s	remaining: 1.82s
690:	learn: 7759.6768251	total: 4.05s	remaining: 1.81s
691:	learn: 7753.5213054	total: 4.06s	remaining: 1.81s
692:	learn: 7752.5058566	total: 4.07s	remaining: 1.8s
693:	learn: 7751.2024331	total: 4.07s	remaining: 1.79s
694:	learn: 7734.1816091	total: 4.08s	remaining: 1.79s
695:	learn: 7732.3808138	total: 4.08s	remaining: 1.78s
696:	learn: 7722.0836723	total: 4.09s	remaining: 1.78s
697:	learn: 7720.3597831	total: 4.09s	remaining: 1.77s
698:	learn: 7706.3830056	total: 4.1s	remaining: 1.76s
699:	learn: 7697.6936018	total: 4.1s	remaining: 1.76s
700:	learn: 7696

850:	learn: 6437.6764233	total: 4.79s	remaining: 839ms
851:	learn: 6429.2311685	total: 4.8s	remaining: 834ms
852:	learn: 6421.8818957	total: 4.8s	remaining: 828ms
853:	learn: 6420.1030110	total: 4.81s	remaining: 822ms
854:	learn: 6407.8779267	total: 4.81s	remaining: 816ms
855:	learn: 6405.5068825	total: 4.82s	remaining: 811ms
856:	learn: 6396.4429289	total: 4.82s	remaining: 805ms
857:	learn: 6387.3998994	total: 4.83s	remaining: 799ms
858:	learn: 6381.6760202	total: 4.83s	remaining: 793ms
859:	learn: 6378.8859084	total: 4.84s	remaining: 788ms
860:	learn: 6365.9159285	total: 4.84s	remaining: 782ms
861:	learn: 6361.8840373	total: 4.85s	remaining: 776ms
862:	learn: 6352.7256926	total: 4.85s	remaining: 770ms
863:	learn: 6343.8818204	total: 4.85s	remaining: 764ms
864:	learn: 6330.6241632	total: 4.86s	remaining: 759ms
865:	learn: 6330.0603928	total: 4.86s	remaining: 753ms
866:	learn: 6322.9865483	total: 4.87s	remaining: 747ms
867:	learn: 6306.1226524	total: 4.87s	remaining: 741ms
868:	learn: 

25445.11615463851


In [16]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [17]:
y_test_log = np.log(y_test)
y_train_log = np.log(y_train)

In [18]:
sr.fit(train,y_train_log)

Learning rate set to 0.040124
0:	learn: 0.3807987	total: 4.15ms	remaining: 4.14s
1:	learn: 0.3717909	total: 8.31ms	remaining: 4.15s
2:	learn: 0.3630719	total: 12.6ms	remaining: 4.18s
3:	learn: 0.3544896	total: 16.7ms	remaining: 4.17s
4:	learn: 0.3456735	total: 21ms	remaining: 4.17s
5:	learn: 0.3378533	total: 25.1ms	remaining: 4.16s
6:	learn: 0.3302085	total: 29.2ms	remaining: 4.14s
7:	learn: 0.3221425	total: 33.3ms	remaining: 4.13s
8:	learn: 0.3144163	total: 37.4ms	remaining: 4.12s
9:	learn: 0.3074784	total: 41.5ms	remaining: 4.11s
10:	learn: 0.3006908	total: 45.6ms	remaining: 4.1s
11:	learn: 0.2944206	total: 49.7ms	remaining: 4.09s
12:	learn: 0.2878641	total: 53.7ms	remaining: 4.08s
13:	learn: 0.2814979	total: 57.8ms	remaining: 4.07s
14:	learn: 0.2750515	total: 62ms	remaining: 4.07s
15:	learn: 0.2695565	total: 66.1ms	remaining: 4.06s
16:	learn: 0.2638731	total: 70.3ms	remaining: 4.07s
17:	learn: 0.2582533	total: 74.5ms	remaining: 4.06s
18:	learn: 0.2532494	total: 78.6ms	remaining: 4.0

171:	learn: 0.0991968	total: 781ms	remaining: 3.76s
172:	learn: 0.0989239	total: 798ms	remaining: 3.81s
173:	learn: 0.0986448	total: 803ms	remaining: 3.81s
174:	learn: 0.0984545	total: 809ms	remaining: 3.81s
175:	learn: 0.0983428	total: 815ms	remaining: 3.81s
176:	learn: 0.0980989	total: 819ms	remaining: 3.81s
177:	learn: 0.0979007	total: 823ms	remaining: 3.8s
178:	learn: 0.0976936	total: 830ms	remaining: 3.8s
179:	learn: 0.0974939	total: 835ms	remaining: 3.8s
180:	learn: 0.0972490	total: 839ms	remaining: 3.79s
181:	learn: 0.0970590	total: 843ms	remaining: 3.79s
182:	learn: 0.0968820	total: 847ms	remaining: 3.78s
183:	learn: 0.0966722	total: 851ms	remaining: 3.78s
184:	learn: 0.0963190	total: 856ms	remaining: 3.77s
185:	learn: 0.0962149	total: 860ms	remaining: 3.76s
186:	learn: 0.0960209	total: 864ms	remaining: 3.75s
187:	learn: 0.0958054	total: 868ms	remaining: 3.75s
188:	learn: 0.0956348	total: 872ms	remaining: 3.74s
189:	learn: 0.0955390	total: 878ms	remaining: 3.74s
190:	learn: 0.0

348:	learn: 0.0754411	total: 1.58s	remaining: 2.94s
349:	learn: 0.0753573	total: 1.58s	remaining: 2.94s
350:	learn: 0.0752653	total: 1.59s	remaining: 2.94s
351:	learn: 0.0751281	total: 1.6s	remaining: 2.94s
352:	learn: 0.0749974	total: 1.6s	remaining: 2.94s
353:	learn: 0.0748587	total: 1.61s	remaining: 2.94s
354:	learn: 0.0747254	total: 1.62s	remaining: 2.94s
355:	learn: 0.0746047	total: 1.63s	remaining: 2.95s
356:	learn: 0.0745187	total: 1.64s	remaining: 2.95s
357:	learn: 0.0744360	total: 1.65s	remaining: 2.95s
358:	learn: 0.0743552	total: 1.66s	remaining: 2.96s
359:	learn: 0.0742332	total: 1.66s	remaining: 2.96s
360:	learn: 0.0742040	total: 1.67s	remaining: 2.96s
361:	learn: 0.0741201	total: 1.68s	remaining: 2.96s
362:	learn: 0.0740002	total: 1.69s	remaining: 2.96s
363:	learn: 0.0738398	total: 1.69s	remaining: 2.96s
364:	learn: 0.0737676	total: 1.7s	remaining: 2.96s
365:	learn: 0.0736804	total: 1.71s	remaining: 2.96s
366:	learn: 0.0735661	total: 1.72s	remaining: 2.96s
367:	learn: 0.0

521:	learn: 0.0602237	total: 2.58s	remaining: 2.36s
522:	learn: 0.0601062	total: 2.58s	remaining: 2.35s
523:	learn: 0.0600750	total: 2.58s	remaining: 2.35s
524:	learn: 0.0599708	total: 2.59s	remaining: 2.34s
525:	learn: 0.0598783	total: 2.6s	remaining: 2.34s
526:	learn: 0.0598658	total: 2.6s	remaining: 2.33s
527:	learn: 0.0597591	total: 2.61s	remaining: 2.33s
528:	learn: 0.0596455	total: 2.61s	remaining: 2.33s
529:	learn: 0.0595890	total: 2.62s	remaining: 2.32s
530:	learn: 0.0594855	total: 2.62s	remaining: 2.31s
531:	learn: 0.0594361	total: 2.63s	remaining: 2.31s
532:	learn: 0.0593965	total: 2.63s	remaining: 2.31s
533:	learn: 0.0593518	total: 2.64s	remaining: 2.3s
534:	learn: 0.0592263	total: 2.64s	remaining: 2.29s
535:	learn: 0.0591679	total: 2.64s	remaining: 2.29s
536:	learn: 0.0590991	total: 2.65s	remaining: 2.29s
537:	learn: 0.0590168	total: 2.65s	remaining: 2.28s
538:	learn: 0.0589232	total: 2.66s	remaining: 2.27s
539:	learn: 0.0588346	total: 2.66s	remaining: 2.27s
540:	learn: 0.0

695:	learn: 0.0500386	total: 3.35s	remaining: 1.46s
696:	learn: 0.0499740	total: 3.36s	remaining: 1.46s
697:	learn: 0.0499675	total: 3.36s	remaining: 1.46s
698:	learn: 0.0499213	total: 3.37s	remaining: 1.45s
699:	learn: 0.0498504	total: 3.37s	remaining: 1.45s
700:	learn: 0.0497711	total: 3.38s	remaining: 1.44s
701:	learn: 0.0497462	total: 3.38s	remaining: 1.44s
702:	learn: 0.0497005	total: 3.39s	remaining: 1.43s
703:	learn: 0.0496521	total: 3.39s	remaining: 1.43s
704:	learn: 0.0495927	total: 3.4s	remaining: 1.42s
705:	learn: 0.0495615	total: 3.4s	remaining: 1.42s
706:	learn: 0.0494821	total: 3.41s	remaining: 1.41s
707:	learn: 0.0494595	total: 3.41s	remaining: 1.41s
708:	learn: 0.0494547	total: 3.42s	remaining: 1.4s
709:	learn: 0.0493849	total: 3.42s	remaining: 1.4s
710:	learn: 0.0493177	total: 3.42s	remaining: 1.39s
711:	learn: 0.0492800	total: 3.43s	remaining: 1.39s
712:	learn: 0.0492151	total: 3.43s	remaining: 1.38s
713:	learn: 0.0491646	total: 3.44s	remaining: 1.38s
714:	learn: 0.04

...

373:	learn: 0.0707594	total: 1.56s	remaining: 2.61s
374:	learn: 0.0706091	total: 1.57s	remaining: 2.61s
375:	learn: 0.0704194	total: 1.57s	remaining: 2.61s
376:	learn: 0.0702791	total: 1.58s	remaining: 2.6s
377:	learn: 0.0702560	total: 1.58s	remaining: 2.6s
378:	learn: 0.0701399	total: 1.59s	remaining: 2.6s
379:	learn: 0.0700252	total: 1.59s	remaining: 2.6s
380:	learn: 0.0700018	total: 1.6s	remaining: 2.6s
381:	learn: 0.0699309	total: 1.6s	remaining: 2.59s
382:	learn: 0.0698765	total: 1.61s	remaining: 2.59s
383:	learn: 0.0698467	total: 1.61s	remaining: 2.58s
384:	learn: 0.0697178	total: 1.61s	remaining: 2.58s
385:	learn: 0.0696037	total: 1.62s	remaining: 2.57s
386:	learn: 0.0694570	total: 1.62s	remaining: 2.57s
387:	learn: 0.0694371	total: 1.63s	remaining: 2.56s
388:	learn: 0.0693138	total: 1.63s	remaining: 2.56s
389:	learn: 0.0692523	total: 1.63s	remaining: 2.56s
390:	learn: 0.0691314	total: 1.64s	remaining: 2.55s
391:	learn: 0.0690055	total: 1.64s	remaining: 2.55s
392:	learn: 0.06895

560:	learn: 0.0566319	total: 2.34s	remaining: 1.83s
561:	learn: 0.0565718	total: 2.35s	remaining: 1.83s
562:	learn: 0.0565118	total: 2.35s	remaining: 1.82s
563:	learn: 0.0564374	total: 2.36s	remaining: 1.82s
564:	learn: 0.0563757	total: 2.36s	remaining: 1.82s
565:	learn: 0.0562873	total: 2.37s	remaining: 1.81s
566:	learn: 0.0562786	total: 2.37s	remaining: 1.81s
567:	learn: 0.0561859	total: 2.38s	remaining: 1.81s
568:	learn: 0.0561105	total: 2.38s	remaining: 1.8s
569:	learn: 0.0560348	total: 2.39s	remaining: 1.8s
570:	learn: 0.0559666	total: 2.39s	remaining: 1.8s
571:	learn: 0.0559181	total: 2.4s	remaining: 1.79s
572:	learn: 0.0558448	total: 2.4s	remaining: 1.79s
573:	learn: 0.0558384	total: 2.4s	remaining: 1.78s
574:	learn: 0.0557372	total: 2.41s	remaining: 1.78s
575:	learn: 0.0557302	total: 2.41s	remaining: 1.78s
576:	learn: 0.0556463	total: 2.42s	remaining: 1.77s
577:	learn: 0.0555780	total: 2.42s	remaining: 1.77s
578:	learn: 0.0554804	total: 2.42s	remaining: 1.76s
579:	learn: 0.0554

721:	learn: 0.0468364	total: 3.13s	remaining: 1.2s
722:	learn: 0.0467784	total: 3.14s	remaining: 1.2s
723:	learn: 0.0466578	total: 3.15s	remaining: 1.2s
724:	learn: 0.0466113	total: 3.16s	remaining: 1.2s
725:	learn: 0.0465202	total: 3.17s	remaining: 1.2s
726:	learn: 0.0464589	total: 3.19s	remaining: 1.2s
727:	learn: 0.0464119	total: 3.19s	remaining: 1.19s
728:	learn: 0.0463119	total: 3.2s	remaining: 1.19s
729:	learn: 0.0462473	total: 3.21s	remaining: 1.19s
730:	learn: 0.0461879	total: 3.22s	remaining: 1.18s
731:	learn: 0.0461615	total: 3.22s	remaining: 1.18s
732:	learn: 0.0460970	total: 3.23s	remaining: 1.18s
733:	learn: 0.0460289	total: 3.24s	remaining: 1.18s
734:	learn: 0.0459618	total: 3.25s	remaining: 1.17s
735:	learn: 0.0458816	total: 3.25s	remaining: 1.17s
736:	learn: 0.0458034	total: 3.27s	remaining: 1.17s
737:	learn: 0.0457768	total: 3.28s	remaining: 1.16s
738:	learn: 0.0457416	total: 3.28s	remaining: 1.16s
739:	learn: 0.0457071	total: 3.29s	remaining: 1.16s
740:	learn: 0.04569

903:	learn: 0.0377677	total: 4.13s	remaining: 438ms
904:	learn: 0.0377055	total: 4.13s	remaining: 434ms
905:	learn: 0.0376281	total: 4.14s	remaining: 430ms
906:	learn: 0.0376226	total: 4.14s	remaining: 425ms
907:	learn: 0.0375952	total: 4.15s	remaining: 421ms
908:	learn: 0.0375320	total: 4.16s	remaining: 416ms
909:	learn: 0.0374344	total: 4.16s	remaining: 411ms
910:	learn: 0.0373859	total: 4.17s	remaining: 407ms
911:	learn: 0.0373158	total: 4.17s	remaining: 403ms
912:	learn: 0.0373097	total: 4.18s	remaining: 398ms
913:	learn: 0.0372428	total: 4.18s	remaining: 394ms
914:	learn: 0.0372018	total: 4.19s	remaining: 389ms
915:	learn: 0.0371742	total: 4.19s	remaining: 384ms
916:	learn: 0.0371560	total: 4.2s	remaining: 380ms
917:	learn: 0.0371376	total: 4.2s	remaining: 375ms
918:	learn: 0.0371029	total: 4.2s	remaining: 370ms
919:	learn: 0.0370585	total: 4.21s	remaining: 366ms
920:	learn: 0.0370246	total: 4.21s	remaining: 361ms
921:	learn: 0.0369742	total: 4.21s	remaining: 357ms
922:	learn: 0.0

StackingRegressor(estimators=[('RandomForestRegressor',
                               RandomForestRegressor(n_estimators=300)),
                              ('CatBoostRegressor',
                               <catboost.core.CatBoostRegressor object at 0x7f9fb1235470>),
                              ('Lasso', Lasso())],
                  final_estimator=Lasso())

In [28]:
y_pred = sr.predict(test)
y_pred

array([12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03065188,
       12.03065188, 12.03065188, 12.03065188, 12.03065188, 12.03

In [20]:
np.exp(y_pred)

array([167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 167820.77665607, 167820.77665607, 167820.77665607,
       167820.77665607, 1

In [22]:
rmse(y_test,np.exp(y_pred))

88270.86347854992

In [29]:
np.log(np.array([1,2,3]))

array([0.        , 0.69314718, 1.09861229])

# Best model: StackingRegressor - 25372.922631329093


In [61]:
def encoder(scaler = 0,degree = 0):
    #init
    Ordinal_Encoder = ['LotShape', 'LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','HeatingQC',
                  'KitchenQual','FireplaceQu','GarageQual','GarageCond']
    Onehot_Encoder = []
    number_feature = []
    for i in df_new_train:
        if(df_new_train[i].dtype == 'object' and i not in Ordinal_Encoder and i not in drop_features):
            Onehot_Encoder.append(i)
        if(df_new_train[i].dtype != 'object'):
            number_feature.append(i)

    # ordinal Encoder
    oe = OrdinalEncoder()
    OrdEncoder_train = oe.fit_transform(df_new_train[Ordinal_Encoder])
    OrdEncoder_test = oe.transform(df_new_test[Ordinal_Encoder])
    
    #one hot encoder
    #ohe = OneHotEncoder(handle_unknown='ignore')
    ohe = OrdinalEncoder()
    #OneEncoder_train = ohe.fit_transform(df_new_train[Onehot_Encoder]).toarray()
    OneEncoder_train = ohe.fit_transform(df_new_train[Onehot_Encoder])
    #OneEncoder_test = ohe.transform(df_new_test[Onehot_Encoder]).toarray()
    OneEncoder_test = ohe.transform(df_new_test[Onehot_Encoder])
    
    # concat
    train = np.concatenate((OrdEncoder_train,OneEncoder_train), axis=1)
    train = np.concatenate((train,np.array(df_new_train[number_feature])), axis=1)
    test = np.concatenate((OrdEncoder_test,OneEncoder_test), axis=1)
    test = np.concatenate((test,np.array(df_new_test[number_feature])), axis=1)
    
    #Standard Scaler
    if(scaler != 0):
        ss = StandardScaler()
        train = ss.fit_transform(train)
        test = ss.transform(test)
    
    #Polynomial Regression
    if(degree > 0):
        poly_reg = PolynomialFeatures(degree=degree)
        train = poly_reg.fit_transform(train)
        test = poly_reg.transform(test)
    
    #return test,train
    return train,test

In [62]:
train,test = encoder()
model5 = CatBoostRegressor()
model5.fit(train,y_train)
print('CatBoostRegressor: ',end='')

ValueError: Found unknown categories ['Membran'] in column 11 during transform

In [20]:
print(mean_squared_error(y_test, model5.predict(test), squared=False))

25907.39504664792


In [26]:
train.shape

(1168, 233)

In [25]:
sorted_idx = model5.feature_importances_.argsort()
plt.barh(boston.feature_names[sorted_idx], rf.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

233